# **Praktikum 2**

## **Langkah 2: Konfigurasi Server**

In [ ]:
import os
import numpy as np
import cv2
import pickle
import tensorflow as tf
from flask import Flask, request, render_template_string
from skimage.feature import hog

app = Flask(__name__)

# Load Model & Scaler
MODEL_PATH = 'day_night_model.h5'
SCALER_PATH = 'scaler.pkl'

try:
    model = tf.keras.models.load_model(MODEL_PATH)
    with open(SCALER_PATH, 'rb') as f:
        scaler = pickle.load(f)
    print("✅ System Loaded Successfully")
except Exception as e:
    print(f"❌ Error loading system: {e}")

def preprocess_image(image_bytes):
    # Decode gambar dari byte stream
    nparr = np.frombuffer(image_bytes, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    
    # Preprocessing (Harus konsisten dengan fase training)
    img = cv2.resize(img, (256, 256))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    hog_feat = hog(gray, orientations=9, pixels_per_cell=(8,8),
                   cells_per_block=(2,2), block_norm='L2-Hys',
                   visualize=False, feature_vector=True)
                   
    # Normalisasi menggunakan scaler yang sudah dilatih
    return scaler.transform(hog_feat.reshape(1, -1))

@app.route('/', methods=['GET'])
def home():
    # Tampilan antarmuka sederhana untuk upload file
    return render_template_string('''
    <div style="text-align:center; padding:50px;">
        <h1>Day vs Night Classifier</h1>
        <form action="/predict" method="post" enctype="multipart/form-data">
            <input type="file" name="file" required><br><br>
            <button type="submit">Prediksi</button>
        </form>
    </div>
    ''')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        file = request.files['file']
        data = preprocess_image(file.read())
        prediction = model.predict(data)[0][0]
        
        # Threshold 0.5 untuk klasifikasi biner
        label = "Day (Siang)" if prediction > 0.5 else "Night (Malam)"
        return f"<h2 style='text-align:center'>Hasil: {label}</h2><center><a href='/'>Kembali</a></center>"
    except Exception as e:
        return f"Error: {e}"

if __name__ == '__main__':
    # Port 7860 adalah standar untuk aplikasi di Hugging Face Spaces
    app.run(host='0.0.0.0', port=7860)

## **Langkah 3: Konfigurasi Dependencies (requirements.txt)**

In [ ]:
flask
tensorflow-cpu
numpy
scikit-learn
scikit-image
opencv-python-headless
gunicorn

## **Langkah 4: Konfigurasi Docker (Dockerfile)**

Dockerfile adalah resep instruksi untuk membangun image Docker. Docker memastikan aplikasi berjalan dalam lingkungan yang terisolasi dan konsisten, terlepas dari sistem operasi komputer host. Instruksi di dalamnya mencakup pemilihan sistem operasi dasar (Python Slim), instalasi dependensi sistem operasi (seperti libgl1 untuk OpenCV), pembuatan pengguna non-root demi keamanan, instalasi dependensi Python, serta perintah untuk menjalankan aplikasi.

## **Langkah 5: Setup Hugging Face Space**

Login ke Hugging Face.

Buat Space baru dengan SDK Docker.

Di menu Settings > Access Tokens, buat token baru dengan izin Write (Write permission) untuk keperluan upload.

## **Link Hugging Face**

https://huggingface.co/spaces/Emkafie/daynight-classifier-emkafie